In [1]:
import psycopg2 as pg2

In [2]:
conn = pg2.connect(database='dvdrental',user='postgres') # this user has no password

In [3]:
cur = conn.cursor()

In [4]:
cur.execute('SELECT * FROM payment')

In [5]:
# fetchall()
# fetchmany()
# fetchone()

data = cur.fetchmany(10)

In [6]:
data[:5]

[(17503,
  341,
  2,
  1520,
  Decimal('7.99'),
  datetime.datetime(2007, 2, 15, 22, 25, 46, 996577)),
 (17504,
  341,
  1,
  1778,
  Decimal('1.99'),
  datetime.datetime(2007, 2, 16, 17, 23, 14, 996577)),
 (17505,
  341,
  1,
  1849,
  Decimal('7.99'),
  datetime.datetime(2007, 2, 16, 22, 41, 45, 996577)),
 (17506,
  341,
  2,
  2829,
  Decimal('2.99'),
  datetime.datetime(2007, 2, 19, 19, 39, 56, 996577)),
 (17507,
  341,
  2,
  3130,
  Decimal('7.99'),
  datetime.datetime(2007, 2, 20, 17, 31, 48, 996577))]

In [7]:
data[0]

(17503,
 341,
 2,
 1520,
 Decimal('7.99'),
 datetime.datetime(2007, 2, 15, 22, 25, 46, 996577))

In [8]:
for dat in data[0]:
    print(dat)

17503
341
2
1520
7.99
2007-02-15 22:25:46.996577


## Investigating the Tables
<br/>
<div>
<p>In every Postgres engine, there are a set of internal tables Postgres uses to manage its entire structure. These tables are located in the Postgres documentation as the groups <a target="_blank" href="https://www.postgresql.org/docs/current/static/information-schema.html">information_schema</a> and <a target="_blank" href="https://www.postgresql.org/docs/current/static/catalogs-overview.html">system catalogs</a>. These contain all the information about data, names of tables, and types stored in a Postgres database. For example, when we use the attribute, <code>cursor.description</code>, it is pulling information from the internal tables to display to the user.</p>
<p>The team at HUD has not given us details about the structure of their database. To proceed, we need to create our own detailed description of what's included. The Postgres table that contains the list of all tables in the database is named <code>tables</code> and is located in the <code>information_schema</code>. From the <a target="_blank" href="https://www.postgresql.org/docs/current/static/infoschema-tables.html">documentation of the <code>information_schema.tables</code></a> we see that there are plenty of columns. Here are the first three:</p>
<table>
<tbody><tr>
<th>Name</th>
<th>Data Type</th>
<th>Description</th>
</tr>
<tr>
<td>table_catalog</td>
<td>sql_identifier</td>
<td>Name of the database that contains the table (always the current database)</td>
</tr>
<tr>
<td>table_schema</td>
<td>sql_identifier</td>
<td>Name of the schema that contains the table</td>
</tr>
<tr>
<td>table_name</td>
<td>sql_identifier</td>
<td>Name of the table</td>
</tr>
</tbody></table>
<p>At this point, we are only concerned with the names of the tables that are in the database. Taking a look at the table description above, there is a column named <code>table_name</code> that exposes this information. Let's query that column and see what we're dealing with.</p></div>

<div class="ScrollAccordion__section">
<ol>
<li>Perform a <code>SELECT</code> query on the <code>information_schema.tables</code> table that:<ul>
<li>Selects the <code>table_name</code> column</li>
<li>Orders the results by <code>table_name</code> using the <code>ORDER BY</code> option.</li>
</ul>
</li>
<li>Use the <code>cursor.fetchall()</code> method to obtain all result from the query and assign them to the variable named <code>table_names</code>.</li>
<li>Print the number of tables using the <code>len()</code> function on <code>table_names</code>.</li>
<li>Loop through <code>table_names</code> and print the name of each table.</li>
</ol></div><div class="MissionLearn__section"></div>

In [9]:
cur.execute("""
    SELECT table_name 
    FROM information_schema.tables
    ORDER BY table_name
""")

In [10]:
table_names = cur.fetchall()

In [11]:
print(len(table_names))

210


In [12]:
for name in table_names:
    print(name)

('_pg_foreign_data_wrappers',)
('_pg_foreign_servers',)
('_pg_foreign_table_columns',)
('_pg_foreign_tables',)
('_pg_user_mappings',)
('actor',)
('actor_info',)
('address',)
('administrable_role_authorizations',)
('applicable_roles',)
('attributes',)
('category',)
('character_sets',)
('check_constraint_routine_usage',)
('check_constraints',)
('city',)
('collation_character_set_applicability',)
('collations',)
('column_domain_usage',)
('column_options',)
('column_privileges',)
('column_udt_usage',)
('columns',)
('constraint_column_usage',)
('constraint_table_usage',)
('country',)
('customer',)
('customer_list',)
('data_type_privileges',)
('domain_constraints',)
('domain_udt_usage',)
('domains',)
('element_types',)
('enabled_roles',)
('film',)
('film_actor',)
('film_category',)
('film_list',)
('foreign_data_wrapper_options',)
('foreign_data_wrappers',)
('foreign_server_options',)
('foreign_servers',)
('foreign_table_options',)
('foreign_tables',)
('information_schema_catalog_name',)
('in

## Working with Schemas

<ol>
<li>Perform a <code>SELECT</code> query on the <code>information_schema.tables</code> table that:<ul>
<li>Selects the <code>table_name</code> column</li>
<li>Use a <code>WHERE</code> statement to filter the query by selecting only rows where <code>table_schema</code> is equal to <code>public</code></li>
<li>Orders the results by <code>table_name</code> using the <code>ORDER BY</code> option</li>
</ul>
</li>
<li>Use the <code>cursor.fetchall()</code> method to obtain all result from the query. Assign them to the variable named <code>table_names</code>.</li>
<li>Print the number of tables using the <code>len()</code> function on <code>table_names</code>.</li>
<li>Loop through <code>table_names</code> and print the name of each table.</li>
</ol>

In [13]:
cur.execute("""
    SELECT table_name 
    FROM information_schema.tables
    WHERE table_schema = 'public'
    ORDER BY table_name
""")

In [15]:
table_names = cur.fetchall()
print(len(table_names))

22


In [16]:
for name in table_names:
    print(name)

('actor',)
('actor_info',)
('address',)
('category',)
('city',)
('country',)
('customer',)
('customer_list',)
('film',)
('film_actor',)
('film_category',)
('film_list',)
('inventory',)
('language',)
('nicer_but_slower_film_list',)
('payment',)
('rental',)
('sales_by_film_category',)
('sales_by_store',)
('staff',)
('staff_list',)
('store',)


## Describing the Tables